
# Uso de SQL diretamente com spark.sql()

- `spark.sql()` é uma função da API do PySpark que permite executar comandos SQL diretamente no engine distribuído do Spark.

| Fonte                                | Como acessar                            |
| ------------------------------------ | --------------------------------------- |
| **Tabelas Delta**                    | `"SELECT * FROM bronze.clientes"`       |
| **Views temporárias**                | Criadas com `createOrReplaceTempView()` |
| **Tabelas externas (Unity Catalog)** | `catalog.schema.tabela`                 |
| **Arquivos CSV/Parquet**             | Registrando como tabela ou view         |

Criando views para uso com SQL:

```python
df = spark.read.parquet("/mnt/silver/clientes")
df.createOrReplaceTempView("clientes_silver")
```


In [0]:

query = """
SELECT 
  * 
FROM 
  clientes 
WHERE 
  score > 700
"""

spark.sql(query)

# Registro de views temporárias e permanentes

## 🟢 1. View Temporária


Uma view temporária existe apenas na sessão atual do Spark. Ela é útil quando você quer acessar um DataFrame via spark.sql() sem salvá-lo em disco. Para criálo basta usar `df.createOrReplaceTempView()`

- Não persiste em disco.
- Só existe enquanto a SparkSession estiver ativa.
- Pode ser substituída com createOrReplaceTempView.

## 🟡 2. View Global Temporária

Uma global temp view é visível em todas as sessões do Spark enquanto o aplicativo estiver ativo e pode ser criada com `df.createOrReplaceGlobalTempView()`

- Namespace fixo: global_temp.
- Útil quando se compartilha dados entre diferentes notebooks no mesmo app.
- Também não persiste após o encerramento da aplicação.

## 🔴 3. View Permanente (Tabela)

Se quiser persistir os dados em disco (e acessar depois da sessão encerrar), você deve salvar como tabela, utilizando `df.write.saveAsTable()`

- Armazenada em disco (geralmente em formato Delta ou Parquet).
- Persiste entre sessões.
- Registrada no metastore (geralmente Hive ou Unity Catalog).

In [0]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
df_iris = spark.createDataFrame(iris_df)
display(df_iris.limit(3))

In [0]:
df_iris.createOrReplaceTempView("global_temp_iris")
# df_iris.createOrReplaceGlobalTempView("global_temp_iris") <- view global, pode ser acessada usando global_temp 

query_iris = """
SELECT 
  * 
FROM 
  global_temp_iris 
WHERE 
  target = 1
"""

display(spark.sql(query_iris).limit(5))